# Prepare Templates
---
### ▶▶ Run All
---

In [7]:
! python -m pip install -r ../requirements.txt

In [8]:
# Add the src directory to the path

import sys

src_path = "../src/"

sys.path.append(src_path)
print(f"📘 Added src directory to the path: {src_path}")

📘 Added src directory to the path: ../src/


In [9]:
%run scrape_uoc.ipynb

📘 Added src directory to the path: ../src/
🔹 Unit of Competency Code: CUADIG311
🔹 Unit Directory Path: ../Units/CUADIG311
🟢 Successfully retrieved XML file.
🟢 Successfully parsed the XML file
🟢 Successfully extracted Unit Title.
🟢 Successfully extracted Elements and Performance criteria.
🟢 Successfully extracted Foundation Skills.
🟢 Successfully extracted Performance Evidence.
🟢 Successfully extracted Knowledge Evidence.
🟢 Successfully extracted Assessment Conditions.
🟢 Successfully saved unit details to: ../Units/CUADIG311/CUADIG311_details.json


In [10]:
# Unit data to be used in template ('uoc' is generated from 'scrape_uoc.ipynb')

default_details = {
    'file_version': 'v1.0',
    'course_code': "22589VIC",
    'course_title': "Certificate III in Emerging Technologies"
}

additional_details = {
    'file_version': default_details["file_version"],
    'course_code': input(f"Enter course code: (default: {default_details["course_code"]})") or default_details["course_code"],
    'course_title': input(f"Enter course title: (default: {default_details["course_title"]})") or default_details["course_title"]
}

unit_context = uoc | additional_details

print(unit_context)

{'unit_code': 'CUADIG311', 'unit_title': 'Prepare video assets', 'elements': [{'index': '1', 'title': 'Identify and evaluate video assets', 'performance_criteria': [{'index': '1.1', 'description': 'Confirm production schedules and deadlines with required personnel '}, {'index': '1.2', 'description': 'Obtain and assess digital video content according to production requirements'}, {'index': '1.3', 'description': 'Identify file details and confirm they meet production requirements'}, {'index': '1.4', 'description': 'Confirm required output file format, codecs for specified bandwidths and applicable video encoding software'}]}, {'index': '2', 'title': 'Organise video assets', 'performance_criteria': [{'index': '2.1', 'description': 'Load video files into applicable video encoding software '}, {'index': '2.2', 'description': 'Confirm duration and quality of video sequences meet production requirements'}, {'index': '2.3', 'description': 'Confirm video and audio codecs meet production require

In [11]:
# Create the unit directory
from utils import unit_path_from_code

unit_path = unit_path_from_code(unit_code)

import os
os.makedirs(unit_path, exist_ok=True)

print(f"🔹 Unit directory initialised: {unit_path}")

🔹 Unit directory initialised: ../Units/CUADIG311


In [12]:
# Prepopulate the Assessment Mapping document

import tempfile
import os
import shutil
from docxtpl import DocxTemplate
from docx import Document

from utils import Paths, FileNames, StopExecution

# Paths to templates (input & output)
jinja_template_path = os.path.join(Paths.Jinja_Templates, FileNames.Assessment_Mapping)
prepopulated_template_path = os.path.join(unit_path, FileNames.Assessment_Mapping)

# Create temporary file
with tempfile.NamedTemporaryFile(delete=False) as temp_file:
    print(f"🔹 Temporary file created at: {temp_file.name}")
    
    shutil.copy2(jinja_template_path, temp_file.name)
    print(f"🔹 Copied jinja template from: {jinja_template_path}")

    # == Prepopulate Text ===
    template = DocxTemplate(temp_file.name)
    template.render(unit_context)
    print(f"🟢 Successfully populated template with: course code, unit code, file version")

    # Save prepopulated template
    try:
        template.save(temp_file.name)
    except OSError as e:
        if e.errno == 13:
            print(f"🔴 Failed to save prepopulated templates")
            print(f"🟠 Ensure the templates files are not currently open")
        raise StopExecution
    except Exception as e:
        raise e
        raise StopExecution
    
    # === Prepopulate Tables ===
    document = Document(temp_file.name)


    # --- Table: Elements & Performance Criteria ---
    table = document.tables[1]
    
    # remove existing rows
    for row in range(len(table.rows) - 1, 1, -1):
        table._tbl.remove(table.rows[row]._tr)
    
    # add elements and performance criteria
    row = 2
    for el in uoc["elements"]:
        # print(el)

        # add a row for each performance criteria
        for pc in el["performance_criteria"]:
            new_row = table.add_row()
            new_row.cells[1].text = f"{pc["index"]} {pc["description"]}"
            # print(pc)
        
        # add element titles
        table.rows[row].cells[0].text = f"{el["index"]}. {el["title"]}"

        # merge element title cells
        start_cell = table.rows[row].cells[0]
        end_cell = new_row.cells[0]
        start_cell.merge(end_cell)

        row += len(el["performance_criteria"])


    # --- Table: Foundational Skills ---
    table = document.tables[2]

    # remove existing rows
    for row in range(len(table.rows) - 1, 1, -1):
        table._tbl.remove(table.rows[row]._tr)

    # add foundational skills
    row = 2
    for fs in uoc["foundational_skills"]:
        # print(el)

        # add a row for each performance criteria
        for desc in fs["descriptions"]:
            new_row = table.add_row()
            new_row.cells[1].text = f"{desc}"
            # print(desc)
        
        # add skill titles
        table.rows[row].cells[0].text = f"{fs["skill"]}"

        # merge skill title cells
        start_cell = table.rows[row].cells[0]
        end_cell = new_row.cells[0]
        start_cell.merge(end_cell)

        row += len(fs["descriptions"])
    

    # --- Table: Performance Evidence ---
    table = document.tables[4]

    # remove existing rows
    for row in range(len(table.rows) - 1, 1, -1):
        table._tbl.remove(table.rows[row]._tr)

    for pe in uoc["performance_evidence"]:
        new_row = table.add_row()
        new_row.cells[0].text = f"{pe}"
    

    # --- Table: Knowledge Evidence ---
    table = document.tables[5]

    # remove existing rows
    for row in range(len(table.rows) - 1, 1, -1):
        table._tbl.remove(table.rows[row]._tr)

    for ke in uoc["knowledge_evidence"]:
        new_row = table.add_row()
        new_row.cells[0].text = f"{ke}"
    

    # --- Table: Assessment Conditions ---
    table = document.tables[6]

    # remove existing rows
    for row in range(len(table.rows) - 1, 1, -1):
        table._tbl.remove(table.rows[row]._tr)

    for ac in uoc["assessment_conditions"]:
        new_row = table.add_row()
        new_row.cells[0].text = f"{ac}"

    
    # --- Table Range of Conditions ---
    # remove from template
    table = document.tables[3]
    table._tbl.getparent().remove(table._tbl)
    

    # --- Save prepopulated template ---
    document.save(prepopulated_template_path)
    print(f"🟢 Prepopulated templates saved to: {unit_path}")

# Clean up the temporary file
try:
    os.remove(temp_file.name)
    print(f"🔹 Temporary file removed: {temp_file.name}")
except Exception as e:
    print(f"🔴 Failed to remove temporary file: {e}")




🔹 Temporary file created at: C:\Users\josh\AppData\Local\Temp\tmpb6r86ndn
🔹 Copied jinja template from: ../Templates/Jinja/VETAssessmentMapping(CurrentUoC).docx
🟢 Successfully populated template with: course code, unit code, file version
🟢 Prepopulated templates saved to: ../Units/CUADIG311
🔹 Temporary file removed: C:\Users\josh\AppData\Local\Temp\tmpb6r86ndn
